<a href="https://colab.research.google.com/github/dudeh534/Dacon_krx_Bean-Bug/blob/main/%08%EC%B5%9C%EC%86%8C%EB%B6%84%EC%82%B0%ED%8F%AC%ED%8A%B8%ED%8F%B4%EB%A6%AC%EC%98%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import os
from pykrx import stock
# 패키지 임포트: numpy, pandas, matplotlib, seaborn, yfinance, scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import scipy.optimize as sco

# seaborn 세팅하기
sns.set()

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 샤프비율 최대화 포트폴리오 가중치 산출 함수 구현
def get_msr_weights(er, cov):

    # 자산개수
    noa = er.shape[0]

    # 가중치 초기값
    init_guess = np.repeat(1/noa, noa)

    # 자산 가중치 상하한
    bounds = ((-1.0, 1.0), ) * noa

    # 레버리지 불가 제약조건
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1}

    # 목적함수
    def neg_sharpe(weights, er, cov):
        r = weights.T @ er
        vol = np.sqrt(weights.T @ cov @ weights)
        return - r / vol

    # 목적함수 : 포트폴리오 변동성
    def port_vol(weights, cov):
        vol = np.sqrt(weights.T @ cov @ weights)
        return vol

    # 최적화 수행
    res = sco.minimize(port_vol,
                       init_guess,
                       args=(cov),
                       method='SLSQP',
                       constraints=(weights_sum_to_1,),
                       bounds=bounds)

    return res.x

train = pd.read_csv('./train.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

train.drop(['종목명', '고가', '저가', '거래량', '시가' ], inplace=True, axis=1)
train = train.pivot(index='일자', columns='종목코드', values='종가')
print(train)


# 수익률 데이터
rets = train.pct_change().fillna(0)

# 기대수익률 계산
er = rets.mean() * 252

# 공분산행렬 계산
cov = rets.cov() * 252

print(rets)


# MSR 포트폴리오 가중치
msr_weights = get_msr_weights(er, cov)

# MSR 포트폴리오 수익률
msr_ret = np.dot(msr_weights, er)

# MSR 포트폴리오 변동성
msr_vol = np.sqrt(np.dot(msr_weights.T, np.dot(cov, msr_weights)))

print(msr_weights)
print('Return = {}'.format(msr_ret))
print('Volatility = {}'.format(msr_vol))

train_data_first = pd.DataFrame()

sample_df = pd.DataFrame({
    '종목코드' : sample_submission['종목코드'][0:len(msr_weights)],
    'weights' : msr_weights
})
sample_df['순위'] = sample_df['weights'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
print(sample_df)
sample_df.to_csv('baseline_submission.csv', index=False)

sample_df.drop(['weights'], inplace=True, axis=1)

sample_df.to_csv('baseline_submission_1.csv', index=False)
